In [1]:
from lib.v2.parameter_estimation import ParemeterEstimation

estimator = ParemeterEstimation(path = "data/growth_full2.csv", sep=";", col_temp="Temp", col_uia="NH4", col_do="DO", col_doc="DOC")

# intial setup
estimator.set_data_for_interpolation(path = "data/biochem.csv")
estimator.set_conditional_parameter(cond_temp=(25, 28, 32, 33), cond_uia=(0, 0.0001, 0.06, 1),
                                cond_do=(4, 6, 9, 10), cond_csc=(0, 0, 0.5, 3))
estimator.set_food_availablelity_data()
estimator.set_growth_paremater(w0=0.05, wn=40, n0=100, sr=0.92)
estimator.set_partial_harvest_parameter(doc=[60, 90, 100], ph=[0.1, 0.1, 0.1], final_doc=120)
estimator.set_pond_data(area=1000)

estimator.fit()

0.0 0.0 1000
0.015501618490785183 1.5501618490785184e-05 1000
0.03127291152733404 3.1272911527334036e-05 1000
0.05492630152808074 5.4926301528080745e-05 1000
0.08450657640199527 8.450657640199527e-05 1000
0.004980556220025529 4.980556220025529e-06 1000
0.004976676546910233 4.976676546910232e-06 1000
0.0049727998959199415 4.9727998959199415e-06 1000
0.26298221657848897 0.00026298221657848896 1000
0.32879686224120025 0.0003287968622412003 1000
0.39697704336328266 0.00039697704336328265 1000
0.4657964383317 0.00046579643833169997 1000
0.004953461890513268 4.953461890513269e-06 1000
0.6194454696942308 0.0006194454696942307 1000
0.004945747760972272 4.945747760972271e-06 1000
0.004941895202387325 4.941895202387325e-06 1000
0.8664285242596792 0.0008664285242596792 1000
0.004934199085891051 4.934199085891051e-06 1000
1.0380295254457534 0.0010380295254457535 1000
1.1246211554418553 0.0011246211554418553 1000
0.004922677377792897 4.922677377792897e-06 1000
1.297575160857953 0.001297575160857953

(0.01659081696531322,
 0.05717191115320036,
 0.06044067149105883,
 0.037663660460190394,
 -0.057731212925218045,
 0.65)

In [ ]:
import numpy as np
T = 70
sr = 0.92
m = -np.log(sr)/T

weight = []
fr = []

for t in range(1, T):
    weight.append(estimator.single_operation(0, t, m, 0.014060733532218126, -0.07501477685121691, -0.07501477685126447, -0.07501477685126119, 0.143592991226218, 0.04991293595578912)[0])
    fr.append(estimator.fr)

# weight = [estimator.single_operation(0, t, m, 0.046844136331559816, 0.09628233481397985, 0.09115163561079387, 0.029125989406487265, 0.05, 0.05) for t in range(T) if t != 0]


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(weight)

In [ ]:
import pandas as pd

df = pd.read_csv("data/growth_full2.csv", sep=";")

In [ ]:
df[df["DOC"] == 1]["Temp"].values[0]

# Class and Method Researh

In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.interpolate import CubicSpline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def normal_trapezoidal(m, suitable_min, suitable_max, optimal_min, optimal_max):
    """
    m: value in t
    """
    if np.isnan(m):
        ret = 0.25
    elif (m < suitable_min) or (m > suitable_max):
        ret = 0
    else:
        ret = min(
            (
                (m - suitable_min) / (optimal_min - suitable_min),
                1,
                (suitable_max - m) / (suitable_max - optimal_max),
            )
        )

    return ret


def left_trapezoidal(m, suitable_min, suitable_max, optimal_max):
    """
    m: value in t
    """
    if np.isnan(m):
        ret = 0.25
    elif (m < suitable_min) or (m > suitable_max):
        ret = 0
    else:
        ret = min((1, (suitable_max - m) / (suitable_max - optimal_max)))

    return ret

class Weight:
    def __init__(self, t0, t, wn, w0, sr,
        alpha, alpha2, alpha3, alpha4, alpha5, alpha6,
        f_uia, f_o2, f_temp, score_csc, feeding_rate
    ):
        self.t0 = t0
        self.t = t
        self.wn = wn
        self.w0 = w0
        self.sr = sr
        
        self.alpha = alpha
        self.alpha2 = alpha2
        self.alpha3 = alpha3
        self.alpha4 = alpha4
        self.alpha5 = alpha5
        self.alpha6 = alpha6

    @staticmethod
    def _f_temp(value, temp_suitable, temp_optimal):
        res = normal_trapezoidal(value, temp_suitable[0], temp_suitable[1], temp_optimal[0], temp_optimal[1])
        return res

    @staticmethod
    def _f_uia(value, uia_suitable, uia_optimal):
        res = left_trapezoidal(value, uia_suitable[0], uia_suitable[1], uia_optimal[0], uia_optimal[1])
        return res

    @staticmethod
    def _f_o2(value, do_suitable, do_optimal):
        res = normal_trapezoidal(value, do_suitable[0], do_suitable[1], do_optimal[0], do_optimal[1])
        return res

    @staticmethod
    def score_csc(value, csc_suitable, csc_optimal):
        pass

    # @staticmethod
    # def feeding_rate(value, csc_suitable, csc_optimal):
    #     pass

    def get_weight(self):
        pass

        

    

In [ ]:
# f_uia, f_o2, f_temp, temperature = source_data(
#     # path = "data/growth_full2.csv",
#     path = None,
#     temp_suitable_min = 25,
#     temp_suitable_max = 33,
#     temp_optimal_min = 28,
#     temp_optimal_max = 32,
#     do_suitable_min = 4,
#     do_suitable_max = 10,
#     do_optimal_min = 6,
#     do_optimal_max = 9,
#     ua_suitable_min = 0.00,
#     ua_suitable_max = 0.16,
#     ua_optimal_min = 0.00,
#     ua_optimal_max = 0.06,
# )

# csc_suitable_min = 0.00
# csc_suitable_max = 5
# csc_optimal_min = 0.00
# csc_optimal_max = 3